In [1]:
import calendar
import os
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
#from pandas.tseries.offsets import BDay
from pandas.tseries.offsets import * 

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.options.display.max_rows = 11
pd.options.display.float_format = '{:.2f}'.format

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"
osd_path = "\\Users\\User\\OneDrive\\Documents\\obsidian-git-sync\\Data\\"

today = date.today()
yesterday = today - timedelta(days=1)
today, yesterday

(datetime.date(2023, 6, 27), datetime.date(2023, 6, 26))

In [2]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
print(f'today: {today}')
to_mmdd = today.strftime('%m-%d')
print(f'Today in mm-dd format: {to_mmdd}')
print(f'yesterday: {yesterday}')

today: 2023-06-27
Today in mm-dd format: 06-27
yesterday: 2023-06-26 00:00:00


In [3]:
one_month_ago = yesterday + DateOffset(days = -28)
print('One month ago: ', one_month_ago, ' ', one_month_ago.day_name())

One month ago:  2023-05-29 00:00:00   Monday


In [4]:
format_dict = {
    'ttl_qty':'{:,}','qty':'{:,}',
    'from_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}','fm_price':'{:.2f}',
    'price':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','pct':'{:,.2f}%',  
}

cols = 'name fm_md to_md days fm_price to_price pct spd avg max_price min_price ttl_qty trend'.split()

### Get past one quarter data

In [5]:
sql = """
SELECT name
FROM buy 
ORDER BY name
"""
buy = pd.read_sql(sql, const)

names = buy['name'].values.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'AH', 'ASK', 'ASP', 'AWC', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'EA', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'KTC', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TMT', 'WHAIR', 'WHART'"

In [6]:
in_p = in_p + ", 'TFFIF', 'BEM'"
in_p

"'AH', 'ASK', 'ASP', 'AWC', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'EA', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'KTC', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TMT', 'WHAIR', 'WHART', 'TFFIF', 'BEM'"

### Bypass this statement if aboce statement is executed

In [8]:
sql = """
SELECT name
FROM orders
ORDER BY trade, name
"""
#buy = pd.read_sql(sql, conlite)

#names = buy['name'].values.tolist()
#in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

"'ASK', 'ASP', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'GVREIT', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TFFIF', 'TMT', 'WHAIR', 'WHART', 'AWC', 'GULF', 'TTB'"

### End of Selection between Buy or Orders

In [7]:
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '%s' AND name IN (%s) 
ORDER BY name, date"""
sql = sql % (one_month_ago, in_p)
print(sql)


SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '2023-05-29 00:00:00' AND name IN ('AH', 'ASK', 'ASP', 'AWC', 'BANPU', 'BCH', 'CPNREIT', 'DIF', 'EA', 'GVREIT', 'ICHI', 'IVL', 'JASIF', 'JMART', 'JMT', 'KCE', 'KTC', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'SCC', 'SENA', 'SINGER', 'STA', 'SYNEX', 'TMT', 'WHAIR', 'WHART', 'TFFIF', 'BEM') 
ORDER BY name, date


In [8]:
price = pd.read_sql(sql, const)
price.sample(5)

,name,date,price,qty,maxp,minp
80,AWC,2023-06-22,4.90,30073820,4.96,4.88
292,JASIF,2023-06-26,7.00,7538055,7.10,7.00
305,JMART,2023-06-14,20.10,16423960,20.50,19.90
469,RCL,2023-06-08,23.80,5187289,23.90,23.50
519,SENA,2023-06-20,3.12,1216616,3.12,3.08


In [9]:
data_path = "../data/"
file_name = "Quarterly-Price-by-Name.csv"
output_file = data_path + file_name
osd_file = osd_path + file_name

price.to_csv(output_file, header=None, index=False)
price.to_csv(osd_file, header=True, index=False)

### Call ruby ruby\crt-summary.rb

In [10]:
%pwd

'C:\\Users\\User\\OneDrive\\A7\\Daily'

In [11]:
os.chdir("C:\\users\\user\\onedrive\\a7")
%pwd

'C:\\users\\user\\onedrive\\a7'

In [12]:
!ruby ruby\\crt-summary.rb

ruby/../Data/Quarterly-Price-by-Name.csv
name|fm_date|to_date|days|fm_price|to_price|diff|pct|spd|avg|max_price|min_price|trend|nbr|ttl_qty
AH|2023-05-29|2023-05-29|1|34.75|34.75|0.0|0.0|0|0|35.25|34.25||2|1,301,080
AH|2023-05-30|2023-05-31|2|37.5|37.75|0.25|0.67|1|0|38.25|35.0|Upward|4|10,661,114
AH|2023-06-01|2023-06-01|1|37.0|37.0|0.0|0.0|0|0|37.75|36.5|Downward|5|2,882,866
AH|2023-06-02|2023-06-07|4|37.25|39.25|2.0|5.37|8|2|39.25|36.25|Upward|8|8,078,969
AH|2023-06-08|2023-06-09|2|36.5|34.5|-2.0|-5.48|-8|-4|37.75|27.75|Downward|10|103,830,264
AH|2023-06-12|2023-06-12|1|35.25|35.25|0.0|0.0|0|0|36.25|34.75|Upward|11|7,101,473
AH|2023-06-13|2023-06-14|2|35.0|34.5|-0.5|-1.43|-2|-1|35.75|33.75|Downward|13|5,935,722
AH|2023-06-15|2023-06-15|1|34.75|34.75|0.0|0.0|0|0|35.25|34.5|Upward|14|1,501,200
AH|2023-06-16|2023-06-21|4|34.5|32.75|-1.75|-5.07|-7|-2|35.0|32.25|Downward|18|8,367,251
AH|2023-06-22|2023-06-22|1|33.25|33.25|0.0|0.0|0|0|33.75|32.25|Upward|19|2,011,971
AH|2023-06-23|2023-06-

PTTGC|2023-06-02|2023-06-02|1|35.75|35.75|0.0|0.0|0|0|36.25|35.25|Upward|447|15,656,993
PTTGC|2023-06-06|2023-06-06|1|35.5|35.5|0.0|0.0|0|0|36.5|35.5|Downward|448|11,809,498
PTTGC|2023-06-07|2023-06-09|3|36.5|38.5|2.0|5.48|8|2|38.5|35.5|Upward|451|68,192,314
PTTGC|2023-06-12|2023-06-12|1|38.25|38.25|0.0|0.0|0|0|38.25|37.5|Downward|452|12,387,581
PTTGC|2023-06-13|2023-06-14|2|38.5|39.25|0.75|1.95|3|1|39.5|37.5|Upward|454|39,186,877
PTTGC|2023-06-15|2023-06-16|2|38.75|38.0|-0.75|-1.94|-3|-2|39.0|37.5|Downward|456|36,387,767
PTTGC|2023-06-19|2023-06-20|2|38.5|38.75|0.25|0.65|1|0|39.0|37.5|Upward|458|24,449,802
PTTGC|2023-06-21|2023-06-27|5|37.75|36.25|-1.5|-3.97|-6|-2|38.5|36.0|Downward|463|45,198,315
RCL|2023-05-29|2023-06-01|4|24.4|23.3|-1.1|-4.51|-11|-3|24.6|23.3|Downward|467|7,250,484
RCL|2023-06-02|2023-06-14|9|23.7|25.0|1.3|5.49|13|1|25.25|23.3|Upward|475|31,347,151
RCL|2023-06-15|2023-06-15|1|24.6|24.6|0.0|0.0|0|0|25.0|24.6|Downward|476|3,201,174
RCL|2023-06-16|2023-06-16|1|24.7|24

In [13]:
os.chdir("C:\\users\\user\\onedrive\\a7\\daily")
%pwd

'C:\\users\\user\\onedrive\\a7\\daily'

In [14]:
file_name   = 'summary.csv'
input_file = data_path + file_name
df = pd.read_csv(input_file, sep=',', index_col=None)
df.sample(5).style.format(format_dict)

,name,from_date,to_date,days,from_price,to_price,diff,pct,spd,avg,max_price,min_price,ttl_qty,trend,nbr
185,NER,2023-06-19,2023-06-21,3,4.76,4.68,-0.08,-1.68%,-4,-2,4.78,4.68,"7,607,117",Downward,417
55,BCH,2023-05-31,2023-06-02,3,18.50,18.30,-0.20,-1.08%,-2,-1,18.90,18.10,"46,451,170",Downward,111
142,JMT,2023-05-31,2023-05-31,1,43.00,43.00,0.00,0.00%,0,0,44.00,39.25,"69,063,646",Upward,319
136,JMART,2023-06-01,2023-06-07,5,20.10,19.20,-0.90,-4.48%,-9,-2,20.50,18.90,"56,896,458",Downward,302
190,ORI,2023-06-02,2023-06-06,3,10.90,10.80,-0.10,-0.92%,-1,-1,11.00,10.80,"3,865,454",Downward,427


In [15]:
df.rename(columns={'from_date':'fm_date','from_price':'fm_price'},inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df['fm_md'] = df['fm_date'].dt.strftime('%m-%d')
df['to_md'] = df['to_date'].dt.strftime('%m-%d')

### Group analysis

In [20]:
mask_p = df['to_md'] == to_mmdd
df[mask_p][cols].sort_values('pct',ascending=False).head().style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
11,AH,06-27,06-27,1,32.25,32.25,0.00%,0,0,32.50,30.75,"3,038,977",Upward
134,JASIF,06-27,06-27,1,7.10,7.10,0.00%,0,0,7.15,7.05,"4,598,383",Upward
278,TMT,06-27,06-27,1,7.30,7.30,0.00%,0,0,7.30,7.20,"36,725",Upward
269,TFFIF,06-27,06-27,1,7.60,7.60,0.00%,0,0,7.65,7.50,"3,048,233",Upward
258,SYNEX,06-27,06-27,1,12.30,12.30,0.00%,0,0,12.40,11.90,"197,848",Upward


In [21]:
name_up = 'CPNREIT'
mask_n = df.name == name_up
df[mask_n][cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
71,CPNREIT,05-29,06-02,5,12.90,12.80,-0.78%,-1,-1,12.90,12.70,"3,062,027",Downward
72,CPNREIT,06-06,06-07,2,12.90,12.90,0.00%,0,0,13.00,12.60,"3,678,090",Upward
73,CPNREIT,06-08,06-08,1,12.80,12.80,0.00%,0,0,12.90,12.80,"795,603",Downward
74,CPNREIT,06-09,06-09,1,13.00,13.00,0.00%,0,0,13.00,12.70,"2,498,945",Upward
75,CPNREIT,06-12,06-21,8,12.90,10.10,-21.71%,-28,-4,13.00,10.10,"46,450,088",Downward
76,CPNREIT,06-22,06-23,2,10.40,11.00,5.77%,6,3,11.00,10.20,"10,290,298",Upward
77,CPNREIT,06-26,06-26,1,10.70,10.70,0.00%,0,0,11.00,10.70,"5,994,644",Downward
78,CPNREIT,06-27,06-27,1,11.40,11.40,0.00%,0,0,11.90,11.20,"12,987,769",Upward


In [22]:
mask_m = df['to_md'] == to_mmdd
df[mask_m][cols].sort_values('pct',ascending=True).head().style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
117,ICHI,06-15,06-27,9,15.00,12.90,-14.00%,-21,-3,15.40,12.70,"92,708,282",Downward
246,STA,06-19,06-27,7,18.80,16.80,-10.64%,-20,-3,19.30,16.70,"50,239,376",Downward
178,MCS,06-14,06-27,10,7.10,6.45,-9.15%,-13,-2,7.15,6.45,"5,713,014",Downward
196,ORI,06-21,06-27,5,10.80,10.00,-7.41%,-8,-2,11.10,10.10,"42,243,717",Downward
126,IVL,06-21,06-27,5,34.50,32.50,-5.80%,-8,-2,35.00,32.25,"61,943,644",Downward


In [23]:
name_down = 'ICHI'
mask_n = df.name == name_down
df[mask_n][cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
108,ICHI,05-29,05-29,1,14.60,14.60,0.00%,0,0,14.80,14.40,"9,951,857",Upward
109,ICHI,05-30,05-30,1,14.40,14.40,0.00%,0,0,14.70,14.30,"9,294,424",Downward
110,ICHI,05-31,06-01,2,14.50,14.70,1.38%,2,1,14.80,14.30,"15,770,816",Upward
111,ICHI,06-02,06-02,1,14.40,14.40,0.00%,0,0,14.70,14.40,"5,019,522",Downward
112,ICHI,06-06,06-07,2,15.00,15.20,1.33%,2,1,15.30,14.40,"34,468,963",Upward
113,ICHI,06-08,06-08,1,15.00,15.00,0.00%,0,0,15.20,14.70,"12,344,947",Downward
114,ICHI,06-09,06-12,2,15.20,15.20,0.00%,0,0,15.40,15.00,"20,189,064",Upward
115,ICHI,06-13,06-13,1,15.00,15.00,0.00%,0,0,15.40,14.90,"11,995,449",Downward
116,ICHI,06-14,06-14,1,15.30,15.30,0.00%,0,0,15.30,15.00,"6,709,864",Upward
117,ICHI,06-15,06-27,9,15.00,12.90,-14.00%,-21,-3,15.40,12.70,"92,708,282",Downward


In [24]:
df_groupby_trend = df.groupby('trend')

In [25]:
# To retrieve one of the created groups
df_up = df_groupby_trend.get_group('Upward')
df_up[cols].sample(5).style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
60,BCH,06-19,06-20,2,18.10,18.20,0.55%,1,0,18.30,17.80,"12,232,759",Upward
222,SCC,06-22,06-22,1,327.00,327.00,0.00%,0,0,328.00,321.00,"1,233,151",Upward
261,TFFIF,06-12,06-14,3,7.60,7.65,0.66%,1,0,7.65,7.50,"15,323,520",Upward
78,CPNREIT,06-27,06-27,1,11.40,11.40,0.00%,0,0,11.90,11.20,"12,987,769",Upward
38,AWC,06-19,06-20,2,4.90,5.05,3.06%,6,3,5.05,4.82,"76,824,656",Upward


In [26]:
# To retrieve one of the created groups
df_dw = df_groupby_trend.get_group('Downward')
df_dw[cols].sample(5).style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
172,MCS,05-29,05-31,3,7.15,7.10,-0.70%,-1,-1,7.35,7.00,"1,146,792",Downward
117,ICHI,06-15,06-27,9,15.00,12.90,-14.00%,-21,-3,15.40,12.70,"92,708,282",Downward
24,ASP,06-12,06-13,2,2.84,2.84,0.00%,0,0,2.88,2.84,"2,478,501",Downward
161,KTC,05-31,06-01,2,51.00,50.25,-1.47%,-3,-2,52.75,50.25,"27,505,801",Downward
42,BANPU,06-01,06-01,1,8.20,8.20,0.00%,0,0,8.25,8.10,"48,276,124",Downward


In [27]:
# there is a function called .agg() and it allows us to specify multiple aggregation functions at once
df.groupby('trend').days.agg(['max', 'count', 'median', 'mean'])

,max,count,median,mean
trend,,,,
Downward,12,147,2.00,2.81
Upward,9,148,1.00,1.85


In [28]:
upward_max = df.groupby('trend').days.max().loc['Upward']
downward_max = df.groupby('trend').days.max().loc['Downward']

In [29]:
df.query('days == @upward_max and trend == "Upward"')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
207,RCL,06-02,06-14,9,23.70,25.00,5.49%,13,1,25.25,23.30,"31,347,151",Upward


In [30]:
df.query('days == @downward_max and trend == "Downward"')[cols]

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
82,DIF,06-09,06-26,12,12.30,10.60,-13.82,-17,-2,12.40,10.00,439470965,Downward


### Individual stock analysis

In [31]:
df.query('name == @name_up')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
71,CPNREIT,05-29,06-02,5,12.90,12.80,-0.78%,-1,-1,12.90,12.70,"3,062,027",Downward
72,CPNREIT,06-06,06-07,2,12.90,12.90,0.00%,0,0,13.00,12.60,"3,678,090",Upward
73,CPNREIT,06-08,06-08,1,12.80,12.80,0.00%,0,0,12.90,12.80,"795,603",Downward
74,CPNREIT,06-09,06-09,1,13.00,13.00,0.00%,0,0,13.00,12.70,"2,498,945",Upward
75,CPNREIT,06-12,06-21,8,12.90,10.10,-21.71%,-28,-4,13.00,10.10,"46,450,088",Downward
76,CPNREIT,06-22,06-23,2,10.40,11.00,5.77%,6,3,11.00,10.20,"10,290,298",Upward
77,CPNREIT,06-26,06-26,1,10.70,10.70,0.00%,0,0,11.00,10.70,"5,994,644",Downward
78,CPNREIT,06-27,06-27,1,11.40,11.40,0.00%,0,0,11.90,11.20,"12,987,769",Upward


In [32]:
df_tmp = price.query('name == @name_up')
df_tmp.describe().round(2)

,price,qty,maxp,minp
count,21.00,21.00,21.00,21.00
mean,12.07,4083688.76,12.29,11.88
std,1.02,4190153.77,0.89,1.10
min,10.10,97082.00,10.50,9.90
25%,11.10,714131.00,11.90,10.90
50%,12.80,2993059.00,12.90,12.60
75%,12.90,5906645.00,12.90,12.70
max,13.00,13675377.00,13.00,12.80


In [33]:
df.query('name == @name_down')[cols].style.format(format_dict)

,name,fm_md,to_md,days,fm_price,to_price,pct,spd,avg,max_price,min_price,ttl_qty,trend
108,ICHI,05-29,05-29,1,14.60,14.60,0.00%,0,0,14.80,14.40,"9,951,857",Upward
109,ICHI,05-30,05-30,1,14.40,14.40,0.00%,0,0,14.70,14.30,"9,294,424",Downward
110,ICHI,05-31,06-01,2,14.50,14.70,1.38%,2,1,14.80,14.30,"15,770,816",Upward
111,ICHI,06-02,06-02,1,14.40,14.40,0.00%,0,0,14.70,14.40,"5,019,522",Downward
112,ICHI,06-06,06-07,2,15.00,15.20,1.33%,2,1,15.30,14.40,"34,468,963",Upward
113,ICHI,06-08,06-08,1,15.00,15.00,0.00%,0,0,15.20,14.70,"12,344,947",Downward
114,ICHI,06-09,06-12,2,15.20,15.20,0.00%,0,0,15.40,15.00,"20,189,064",Upward
115,ICHI,06-13,06-13,1,15.00,15.00,0.00%,0,0,15.40,14.90,"11,995,449",Downward
116,ICHI,06-14,06-14,1,15.30,15.30,0.00%,0,0,15.30,15.00,"6,709,864",Upward
117,ICHI,06-15,06-27,9,15.00,12.90,-14.00%,-21,-3,15.40,12.70,"92,708,282",Downward


In [34]:
df_tmp = price.query('name == @name_down')
df_tmp.describe().round(2)

,price,qty,maxp,minp
count,21.00,21.00,21.00,21.00
mean,14.53,10402532.76,14.80,14.31
std,0.63,5285865.11,0.53,0.61
min,12.90,4376808.00,13.20,12.70
25%,14.40,6246901.00,14.60,14.00
50%,14.50,9294424.00,14.80,14.40
75%,15.00,14035253.00,15.30,14.90
max,15.30,24673805.00,15.40,15.00


In [35]:
mask_p = df['to_md'] == to_mmdd
summary = df[mask_p][cols].sort_values(['pct','trend'],ascending=[False,False])

In [36]:
file_name = 'summary-240.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
one_file = one_path + file_name
osd_file = osd_path + file_name

summary[cols].to_csv(output_file, header=True, index=False)
summary[cols].to_csv(data_file, header=True, index=False)
summary[cols].to_csv(one_file, header=True, index=False)
summary[cols].to_csv(osd_file, header=True, index=False)

In [37]:
sql = """
SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '%s' 
ORDER BY name, date"""
sql = sql % (today)
print(sql)


SELECT name, date, price, qty, maxp, minp
FROM price 
WHERE date >= '2023-06-27' 
ORDER BY name, date


In [38]:
df = pd.read_sql(sql, const)
df

,name,date,price,qty,maxp,minp
0,ACE,2023-06-27,1.88,11066943,1.91,1.87
1,ADVANC,2023-06-27,210.00,3277070,213.00,210.00
2,AEONTS,2023-06-27,185.00,86402,187.00,185.00
3,AH,2023-06-27,32.25,3038977,32.50,30.75
4,AIE,2023-06-27,1.30,815426,1.38,1.30
...,...,...,...,...,...,...
187,WHAIR,2023-06-27,7.15,521938,7.15,7.10
188,WHART,2023-06-27,10.40,1488310,10.60,10.40
189,WHAUP,2023-06-27,3.64,1763485,3.72,3.64
190,WICE,2023-06-27,8.20,827660,8.35,8.10
